## Problem Statement

The Conversation AI team, a research initiative founded by Jigsaw and Google builds a technology to prevent voices in Conversation. In 2020, Jigsaw organized a competition on Kaggle where the competitors has to build machine learning models that can identify toxicity in Online conversations, where toxicity is defined as `anything rude, disrespectful, or otherwise likely` to make someone leave the discussion. If these contributions can be identified, we could have a safer, more collaborative internet.     

## Dataset Description

As part of the competition, competitors were provided several files, specifically:

`jigsaw-toxic-comment-train.csv` - data from the [Jigsaw toxic comment classification competition](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge). The dataset is made up of English comments from Wikipedia’s talk page edits.

`jigsaw-unintended-bias-train.csv` - data from the [Jigsaw Unintended Bias in Toxicity Classification competition](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification). This is an expanded version of the Civil Comments dataset with a range of additional labels.

`sample_submission.csv` - a sample submission file.

`test.csv` - comments from Wikipedia talk pages in different non-English languages.

`validation.csv` - comments from Wikipedia talk pages in different non-English languages

## Evaluation Metric
Submissions were evaluated based on Area Under the ROC Curve between the predicted probability and the observed target.

## Strategies to Tackle
### `Monolingual Approach`
Monolingual models are the type of language models which are trained on a single language.

They are focused on understanding and generating text in a specific language. 

For example, a monolingual model trained on English language will be proficient in understanding and generating English text. 

These models are typically for tasks such as text classification, sentiment analysis and more within a specific language.

Monolingual models can be beneficial to utilize when we have a specific language in our training, testing datasets and in the upcoming unseen data. 

### `Multilingual Approach`
Multilingual models are, on the other hand, are trained on multiple different languages. 

They are designed to handle and process text in multiple languages, allowing them to perform across different languages. 

Multilingual models have the advantage of of being able to provide language-agnostic solutions, as they can handle a wide-range of languages. 

They can be used for zero-shot and few-shot learning, where the model can perform a task in a language it has not been seen specifically trained on by leveraging its knowledge of other languages.

### `Which models to use for our problem?`
As per the dataset given in the competition, we have only english data in our training dataset and very samples are given in the validation dataset containing languages `Spanish`, `Turkish` and `Italian` only and the Testing dataset contains languages `Turkish`, `Spanish`, `Italian`, `Russian`, `French` and `Portugese`.

Since in our validation and test dataset contains non-english languages it would be better approach to build multilingual models rather than monolingual models. 

Now, if we had only one language (as stated above) building monolingual models would be a better choice. 

Let's discuss Multilingual models approach a bit more:

How are multilingual models are trained?

Multilingual models are pre-trained on a mix of different languages and they don't distinguish between the languages. 

The English BERT was pre-trained on English Wikipedia and BookCorpus dataset, while multilingual models like mBERT was pre-trained on 102 different languages from largest Wikipedia dataset and XLM-Roberta was pre-trained on CommonCrawl dataset from 100 different languages respectively.

### `Cross Lingual Transfer`
Cross-lingual transfer refers to transfer learning using data and models available for one language for which ample such resources are available (e.g., English) to solve tasks in another, commonly more low-resource, language.

In our case, we are trying to create an application that can automatically detect whether a sentence or phrase is toxic or not. 

Models like XLM-Roberta provides us the ability to fine tune it on English dataset and predict to identify comments in any different language.

XLM-R is able to take it's specific knowledge learnt in one language and apply it to a different langauge (languages), even though it never seen the language during fine-tuning. 

This concept is of transfer learning applied from one language to another language is referred to as `Cross-Lingual Transfer (AKA Zero-Shot Learning)` .

Another reason to use Pre-Trained multi-lingual models for a task like this (as in our case) is that is the `Lack of languages by resources` i.e., different languages have different amounts of training data available to build models like BERT and its variants. 

Some languages like English, Chinese, Russian, Indonesian, Vietnamese etc. are the languages that have high resource languages, whereas languages like sundanese, assamese etc. are low resource languages. 

Training our own BERT like model on these low resources could be very expensive in terms of data collection and performance-wise  , therefore, We should leverage these multi-lingual models.

### `What experiments did I perform ?`
At the Overall level, I performed 9 experiments with the following ideas keeping in mind.

Perform Pre-processing techniques like removal of stopwords, removing URLs, Contraction to Expansion of words, removing multiple characters from words and removal of punctuations.

From model stand point we experimented with 2 models mBERT & XLM-Roberta.

From Training dataset perspective we used 2 types of datasets: one case where we used the provided training datasets where we tried to balance the dataset by the target and the other case where we used the translated training dataset of languages provided in the test dataset along with the english language with class balancing.

We always trained on validation dataset as well to further boost the performance of the model.

Now we build a model with the following ideas:
> Training on original training & validation datasets, class balancing (undersampling), fine-tuning the model for 2 epochs on training as well as validation dataset, and will not perform any preprocessing dataset. We will be leveraging the TPUs offered by Kaggle.

## Installing Libraries

In [1]:
!pip install nltk
!pip install transformers --quiet

import re
import nltk
import string
import os, gc
import pandas as pd
import tensorflow as tf
from transformers import TFAutoModel
from transformers import AutoTokenizer
nltk.download('stopwords')

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


D0506 05:34:07.119779858    4677 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0506 05:34:07.119807585    4677 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0506 05:34:07.119811295    4677 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0506 05:34:07.119814245    4677 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0506 05:34:07.119816782    4677 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0506 05:34:07.119819332    4677 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0506 05:34:07.119822001    4677 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0506 05:34:07.

True

## Setting data paths

In [2]:
main_data_dir_path = "../input/jigsaw-multilingual-toxic-comment-classification/"
toxic_comment_train_csv_path = main_data_dir_path + "jigsaw-toxic-comment-train.csv"
unintended_bias_train_csv_path = main_data_dir_path + "jigsaw-unintended-bias-train.csv"
validation_csv_path = main_data_dir_path + "validation.csv"
test_csv_path = main_data_dir_path + "test.csv"
submission_csv_path = main_data_dir_path + "sample_submission.csv"

## TPU Configurations

Intializing the TPU configurations and other constants like `number of epochs, batch_size (16 * number of cores offered on TPUS), MAX_LEN (length of the sentence), we use xlm-roberta-large model, number of samples (for undersampling) = 150k, Learning_rate = 1e-5 etc.`

In [3]:
#################### TPU Configurations ####################
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

AUTO = tf.data.experimental.AUTOTUNE
# Configuration
EPOCHS = 2
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192
MODEL = 'xlm-roberta-large'
NUM_SAMPLES = 150000
RANDOM_STATE = 42
LEARNING_RATE = 1e-5 ######################### MAIN CHANGE ############################
WEIGHT_DECAY = 1e-6

Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/

## Reading & Balancing the data by Target column

In [4]:
## Reading csv files 
train1 = pd.read_csv(toxic_comment_train_csv_path)
train2 = pd.read_csv(unintended_bias_train_csv_path)
valid = pd.read_csv(validation_csv_path)
test = pd.read_csv(test_csv_path)
sub = pd.read_csv(submission_csv_path)

## Converting floating points to integers ##
train2.toxic = train2['toxic'].round().astype(int)

##### BALANCING THE DATA ##### 
# : Taking all the data from toxic_comment_train_file & all data corresponding to unintended bias train file
# & sampling 150k observations randomly from non-toxic observation population.

# Combine train1 with a subset of train2
train = pd.concat([
    train1[['comment_text', 'toxic']],
    train2[['comment_text', 'toxic']].query('toxic==1'),
    train2[['comment_text', 'toxic']].query('toxic==0').sample(n=NUM_SAMPLES, random_state=RANDOM_STATE)
])

## Dropping missing observations with respect to comment-text column 
train = train.dropna(subset=['comment_text'])

In [5]:
def encode(texts, tokenizer, max_len):
    """
    Function takes a list of texts, tokenizer (object)
    initialized from HuggingFace library, max_len (defines
    of how long the sentence lengths should be).
    """       
    tokens = tokenizer(texts, max_length=max_len, 
                    truncation=True, padding='max_length',
                    add_special_tokens=True, return_tensors='np')
    
    return tokens

## Encoding comment_text

We first initialize the tokenizer from Hugging Face transformer library and encoding our training, validation and test dataset comment_texts. 

In [6]:
## Intializing the tokenizer ##
tokenizer = AutoTokenizer.from_pretrained(MODEL)

train_inputs = encode(train['comment_text'].values.tolist(), 
                      tokenizer, max_len=MAX_LEN)
validation_inputs = encode(valid['comment_text'].values.tolist(),
                          tokenizer, max_len=MAX_LEN)
test_inputs = encode(test['content'].values.tolist(),
                    tokenizer, max_len=MAX_LEN)

## Preparing data using tf.data.Data API

Writing a function to create a tuple of inputs and outputs, where inputs have a dictionary datatype.

We'll be leveraging tf.data.Data API to pass our inputs and outputs as tuple, i.e., (inputs, outputs), inputs are `{"input_ids": input_ids, "attention_mask": attention_mask} and outputs labels`.

In [7]:
def map_fn(input_ids, attention_mask, labels=None):
    if labels is not None:
        return {"input_ids": input_ids, "attention_mask": attention_mask}, labels
    else:
        return {"input_ids": input_ids, "attention_mask": attention_mask}

In [8]:
#### Creating the training dataset ####
train_dataset = tf.data.Dataset.from_tensor_slices((train_inputs["input_ids"],
                                                    train_inputs["attention_mask"],
                                                   train['toxic']))
train_dataset = train_dataset.map(map_fn)
train_dataset = train_dataset.repeat().shuffle(buffer_size=2048,seed=RANDOM_STATE).batch(BATCH_SIZE).prefetch(AUTO)

#### Creating the validation dataset ####
validation_dataset = tf.data.Dataset.from_tensor_slices((validation_inputs['input_ids'],
                                                         validation_inputs['attention_mask'],
                                                        valid['toxic']))
validation_dataset = validation_dataset.map(map_fn)
validation_dataset = validation_dataset.batch(BATCH_SIZE).prefetch(AUTO)

#### Creating the testing dataset ####
test_dataset = tf.data.Dataset.from_tensor_slices((test_inputs['input_ids'],
                                                  test_inputs['attention_mask']))
test_dataset = test_dataset.map(map_fn)
test_dataset = test_dataset.batch(BATCH_SIZE)

## Building the model

In [9]:
def build_model(transformer_layer, max_len):
    """
    Creating the model input layers, output layers,
    model definition and compilation.
        
    Returns: model object after compiling. 
    """
    input_ids = tf.keras.layers.Input(shape=(max_len,), 
                                      dtype=tf.int32, 
                                      name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(max_len,), 
                                       dtype=tf.int32, 
                                       name="attention_mask")
    output = transformer_layer.roberta(input_ids, ############# NOTE THAT this .roberta property is very important to use.
                                       ###### otherwise we'll get a nasty error while loading our saved model.   
                                 attention_mask=attention_mask)[1]
    x = tf.keras.layers.Dense(1024, activation='relu')(output)
    y = tf.keras.layers.Dense(1, activation='sigmoid',name='outputs')(x)
    model = tf.keras.models.Model(inputs=[input_ids, attention_mask],
                             outputs=y)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE,
                                         weight_decay=WEIGHT_DECAY)
    loss = tf.keras.losses.BinaryCrossentropy()
    AUC = tf.keras.metrics.AUC()
    
    model.compile(loss=loss, metrics=[AUC], optimizer=optimizer)    
    return model

## Loading model on TPUs

It is important to initialize & compile the model inside the `with strategy.scope()`.

One thing I want to point out that for some reason I was getting different results even though I was setting the seed before initializing the model, but the results are always consistent even though the results differ very little every time we run the pipeline.

In [10]:
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    tf.random.set_seed(RANDOM_STATE)
    model = build_model(transformer_layer,
                        max_len=MAX_LEN)
model.summary()

All model checkpoint layers were used when initializing TFXLMRobertaModel.

All the layers of TFXLMRobertaModel were initialized from the model checkpoint at xlm-roberta-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 192)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 192)]        0           []                               
                                                                                                  
 roberta (TFXLMRobertaMainLayer  TFBaseModelOutputWi  559890432  ['input_ids[0][0]',              
 )                              thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 192,                                           

## Training the model on Only English data for 2 epochs

In [11]:
train_steps_per_epoch = train_inputs['input_ids'].shape[0] // BATCH_SIZE
train_history = model.fit(train_dataset,
                         steps_per_epoch=train_steps_per_epoch,
                         validation_data=validation_dataset,
                         epochs=2) 

Epoch 1/2


/usr/local/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:459: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  warnings.warn(
2023-05-06 05:38:32.900327: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_790/ReadVariableOp.
2023-05-06 05:38:35.393659: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_790/ReadVariableOp.


3795/3795 [==============================] - ETA: 0s - loss: 0.0547 - auc: 0.9970

2023-05-06 06:03:39.886988: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-05-06 06:03:40.421135: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


3795/3795 [==============================] - 1623s 379ms/step - loss: 0.0547 - auc: 0.9970 - val_loss: 0.3265 - val_auc: 0.9095
Epoch 2/2
3795/3795 [==============================] - 1402s 369ms/step - loss: 0.0448 - auc: 0.9980 - val_loss: 0.3493 - val_auc: 0.8918


## Training the model on Validation data for 2 epochs further to fine-tune on it

In [12]:
validation_steps_per_epoch = validation_inputs['input_ids'].shape[0] // BATCH_SIZE
validation_history = model.fit(validation_dataset.repeat(),
                              steps_per_epoch=validation_steps_per_epoch,
                              epochs=2)

Epoch 1/2
62/62 [==============================] - 23s 367ms/step - loss: 0.2313 - auc: 0.9276
Epoch 2/2
62/62 [==============================] - 116s 367ms/step - loss: 0.1499 - auc: 0.9719


## Submit to Competition

In [13]:
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub.to_csv('submission.csv', index=False)

2023-05-06 06:30:25.204263: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-05-06 06:30:25.715181: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


499/499 [==============================] - 80s 119ms/step


In [14]:
sub.head()

,id,toxic
0,0,0.000168
1,1,0.000202
2,2,0.157186
3,3,0.000174
4,4,0.000155


- Public LeaderBoard score on kaggle (test dataset): 0.936 and Private LeaderBoard score : 0.9346

## Results
| Experiment | Public Test LeaderBoard Score | Private Test LeaderBoard Score |
| --- | --- | --- |
| 1 (mBERT + No Preprocessing + BCE Loss + Fine tune on original training and validation datasets for 2 epochs each + Learning_rate = 2e-5) | 0.8850 | 0.8869 |
|2 (xlm-roberta-large + No Preprocessing + BCE Loss + Fine tune on original training and validation datasets for 2 epochs each + Learning_rate = 2e-5) | 0.9259 | 0.9264 |
|3 (mBERT + Preprocessing + BCE Loss + Fine tune on original training and validation datasets for 2 epochs each + Learning_rate = 2e-5) | 0.8259 | 0.8239 |
|4 (xlm-roberta-large + Preprocessing + BCE Loss + Fine tune on original training and validation datasets for 2 epochs each + Learning_rate = 2e-5) | 0.8755 | 0.8754 |
|5 (mBERT + No Preprocessing + BCE Loss + Fine tune on translated in languages present in test (along with english original english) training and validation datasets for 2 epochs each + Learning_rate = 1e-5) |  0.9195 | 0.9212 |
|6 ((xlm-roberta-large + No Preprocessing + BCE Loss + Fine tune on translated in languages present in test (along with english original english) training and validation datasets for 2 epochs each + Learning_rate = 1e-5) |  0.9329 | 0.9212 |
|7 (mBERT + Preprocessing + BCE Loss + Fine tune on translated in languages present in test (along with english original english) training and validation datasets for 2 epochs each + Learning_rate = 1e-5) |  0.8696 | 0.9212 |
|8 ((xlm-roberta-large + Preprocessing + BCE Loss + Fine tune on translated in languages present in test (along with english original english) training and validation datasets for 2 epochs each + Learning_rate = 1e-5) |  0.8861 | 0.8866 |
|9 (xlm-roberta-large + No Preprocessing + BCE Loss + Fine tune on original training and validation datasets for 2 epochs each + Learning_rate = 1e-5) | 0.936 | 0.9346 |

## Saving the model

In [15]:
model_save_path = "../working/Multilingual_toxic_comment_classifier"
model.save(model_save_path)

INFO:tensorflow:Assets written to: ../working/Multilingual_toxic_comment_classifier/assets


INFO:tensorflow:Assets written to: ../working/Multilingual_toxic_comment_classifier/assets


## Loading the model 

In [16]:
loaded_model = tf.keras.models.load_model(model_save_path)
y = loaded_model.predict(test_dataset.take(1))
y[:6]

Writing the function to prepare for the new text, we encode the text using the `tokenizer with the sentence length=192`

In [53]:
from transformers import AutoTokenizer
tokenizer_ = AutoTokenizer.from_pretrained("xlm-roberta-large")

text = "politicians are like cancer for this country"
def prep_data(text, tokenizer, max_len=192):
    tokens = tokenizer(text, max_length=max_len, 
                    truncation=True, padding='max_length',
                    add_special_tokens=True, return_tensors='tf')
    
    return {"input_ids": tokens['input_ids'],
            "attention_mask": tokens['attention_mask']}

Predicting the probability of toxic and non-toxic on a sample text.

In [56]:
prob_of_toxic_comment = loaded_model.predict(prep_data(text=text, tokenizer=tokenizer_, max_len=192))[0][0]
prob_of_non_toxic_comment = 1 - prob_of_toxic_comment
prob_of_toxic_comment, prob_of_non_toxic_comment
probs = {"prob_of_toxic_comment": prob_of_toxic_comment,
 "prob_of_non_toxic_comment": prob_of_non_toxic_comment}
probs

1/1 [==============================] - 0s 500ms/step


{'prob_of_toxic_comment': 0.368846,
 'prob_of_non_toxic_comment': 0.6311540007591248}